In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
170,ABW,North America,Aruba,2020-09-04,2292.0,81.0,63.429,13.0,1.0,0.714,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
419,AFG,Asia,Afghanistan,2020-09-04,38288.0,45.0,22.714,1410.0,1.0,1.286,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
586,AGO,Africa,Angola,2020-09-04,2777.0,48.0,51.714,112.0,3.0,1.000,...,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-03,France,297166.0,0.0,30727.0,0.0,0.0,0.0
0,2020-09-03,China,89978.0,0.0,4733.0,0.0,0.0,0.0
0,2020-09-03,Italy,272415.0,0.0,35524.0,0.0,0.0,0.0
0,2020-09-03,Spain,486482.0,0.0,29248.0,0.0,0.0,0.0
0,2020-09-03,United States,6198374.0,0.0,186929.0,0.0,0.0,0.0
0,2020-09-03,World,26538302.0,0.0,871367.0,0.0,0.0,0.0
0,2020-09-03,United Kingdom,340019.0,0.0,41552.0,0.0,0.0,0.0
0,2020-09-03,Germany,247274.0,0.0,9336.0,0.0,0.0,0.0
0,2020-09-03,Iran,382896.0,0.0,22039.0,0.0,0.0,0.0
0,2020-09-03,Turkey,274587.0,0.0,6500.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-04,France,297166.0,300181.0,30727.0,30706.0,-3015.0,21.0
0,2020-09-04,China,89978.0,89986.0,4733.0,4735.0,-8.0,-2.0
0,2020-09-04,Italy,272415.0,272912.0,35524.0,35507.0,-497.0,17.0
0,2020-09-03,Spain,486482.0,488513.0,29248.0,29234.0,-2031.0,14.0
0,2020-09-04,United States,6198374.0,6150655.0,186929.0,186797.0,47719.0,132.0
0,2020-09-04,World,26538302.0,26335685.0,871367.0,869209.0,202617.0,2158.0
0,2020-09-04,United Kingdom,340019.0,340411.0,41552.0,41527.0,-392.0,25.0
0,2020-09-04,Germany,247274.0,246948.0,9336.0,9319.0,326.0,17.0
0,2020-09-04,Iran,382896.0,380746.0,22039.0,21926.0,2150.0,113.0
0,2020-09-04,Turkey,274587.0,274943.0,6500.0,6511.0,-356.0,-11.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")